# <div style="color:white;display:inline-block;border-radius:5px;background-color:#FF7F50;font-family:Nexa;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:80%;letter-spacing:0.5px;margin:0"><b> </b> Radiological Society of North America (RSNA) 2023 Abdominal Trauma Detection Challenge</p></div>

#### **Introduction**
Traumatic injury is the most common cause of death in the first four decades of life and a major  public health problem around the world. There are estimated to be around 5 million annual deaths worldwide from traumatic injury. Prompt and accurate diagnosis of traumatic injury is crucial for initiating appropriate and timely interventions, which can significantly improve patient outcomes and survival rates. Computed Tomography (CT) has become an indispensable tool in evaluating patients with suspected abdominal injuries due to its ability to provide detailed cross-sectional images of the abdomen.

Abdominal trauma often cannot be diagnosed clinically by physical exam, patient symptoms, or laboratory tests. Interpreting CT scans for abdominal trauma, however, can be a complex and time-consuming task, especially when multiple injuries or areas of subtle active bleeding are present.

The goal of the competition is to identify several potential injuries in CT scans of trauma patients. Any of these injuries can be fatal on a short time frame if untreated so there is great value in rapid diagnosis.

#### **Common terms**
- **bowel_injury** - Damage or harm that occurs to the intestines.
- **bowel_healthy** - No damage or harm to the intestines.
- **extravasation_injury** - Refers to internal bleeding.
- **extravasation_healthy** - No internal bleeding detected.
- **kidney_healthy** - No kidney injury detected
- **kidney_low** - Low grade Kidney injury detected
- **kidney_high** - High grade Kidney injury detected
- **liver_healthy** - No liver injury detected
- **liver_low** - Low grade liver injury detected
- **liver_high** - High grade liver injury detected
- **spleen_healthy** - No spleen injury detected
- **spleen_low** - Low grade spleen injury detected
- **spleen_high** - High grade spleen injury detected

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#FF7F50;font-family:Nexa;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:80%;letter-spacing:0.5px;margin:0"><b> </b>Installing & Importing Modules</p></div>

MONAI is a medical imaging framework in Python that provides tools for deep-learning based medical image analysis.

In [1]:
import warnings
warnings.filterwarnings('ignore')

!pip install monai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00


"dicom2nifti" converts DCM (DICOM) images to NIfTI (Neuroimaging Informatics Technology Initiative) format.

In [2]:
!pip install dicom2nifti

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 62.4 MB/s eta 0:00:00


In [3]:
import logging
import torchvision
import os
import sys
import shutil
import tempfile
import pydicom
from glob import glob
import dicom2nifti
import matplotlib.pyplot as plt
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import pandas as pd
import nibabel as nib
import monai
from nilearn import plotting
from monai.config import print_config
from monai.data import DataLoader, ImageDataset, CSVSaver
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    Orientation,
    ScaleIntensityRange
)
from monai.utils import set_determinism
from PIL import Image
import dicom2nifti.settings as settings

pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

# /kaggle/input/rsna-2023-abdominal-trauma-detection/

MONAI version: 1.2.0
Numpy version: 1.23.5
Pytorch version: 2.0.0+cpu
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: /opt/conda/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.12
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: 0.21.0
Pillow version: 9.5.0
Tensorboard version: 2.12.3
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.1+cpu
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.3
pandas version: 1.5.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: 4.30.2
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommende

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#FF7F50;font-family:Nexa;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:80%;letter-spacing:0.5px;margin:0"><b> </b> EDA on "train.csv"</p></div>

Here i'm identifying patients having only one of the eight injuries. Their series files will be converted to nifti files and used for training, testing and evaluation.

In [4]:
df = pd.DataFrame(pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection//train.csv'))

Columns names

In [5]:
df.columns

Index(['patient_id', 'bowel_healthy', 'bowel_injury', 'extravasation_healthy',
       'extravasation_injury', 'kidney_healthy', 'kidney_low', 'kidney_high',
       'liver_healthy', 'liver_low', 'liver_high', 'spleen_healthy',
       'spleen_low', 'spleen_high', 'any_injury'],
      dtype='object')

Checking datatypes

In [6]:
df.dtypes

patient_id               int64
bowel_healthy            int64
bowel_injury             int64
extravasation_healthy    int64
extravasation_injury     int64
kidney_healthy           int64
kidney_low               int64
kidney_high              int64
liver_healthy            int64
liver_low                int64
liver_high               int64
spleen_healthy           int64
spleen_low               int64
spleen_high              int64
any_injury               int64
dtype: object

Changing datatype for all columns to boolean

In [7]:
for column in df.columns:
    if column != 'patient_id':
        df[column] = df[column].astype(bool)

Review changes

In [8]:
df.dtypes

patient_id               int64
bowel_healthy             bool
bowel_injury              bool
extravasation_healthy     bool
extravasation_injury      bool
kidney_healthy            bool
kidney_low                bool
kidney_high               bool
liver_healthy             bool
liver_low                 bool
liver_high                bool
spleen_healthy            bool
spleen_low                bool
spleen_high               bool
any_injury                bool
dtype: object

Dataframe shape

In [9]:
# number of rows and columns
number_of_columns = df.shape[1]
number_of_rows = df.shape[0]
print(f'Number of columns: {number_of_columns}')
print(f'Number of rows: {number_of_rows}')

Number of columns: 15
Number of rows: 3147


Patients with no injuries

In [10]:
everything_healthy = df[(df['extravasation_healthy'] == True) & (df['bowel_healthy'] == True) & 
                                           (df['kidney_healthy'] == True) & (df['liver_healthy'] == True) &
                                            (df['spleen_healthy'] == True)]
print(f'There are {everything_healthy.shape[0]} such rows')
print('This means about 80% of the patients in \'train_images\' have no injury.')
everything_healthy

There are 2292 such rows
This means about 80% of the patients in 'train_images' have no injury.


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
1,10005,True,False,True,False,True,False,False,True,False,False,True,False,False,False
2,10007,True,False,True,False,True,False,False,True,False,False,True,False,False,False
3,10026,True,False,True,False,True,False,False,True,False,False,True,False,False,False
6,10082,True,False,True,False,True,False,False,True,False,False,True,False,False,False
7,10104,True,False,True,False,True,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,9835,True,False,True,False,True,False,False,True,False,False,True,False,False,False
3141,9860,True,False,True,False,True,False,False,True,False,False,True,False,False,False
3142,9951,True,False,True,False,True,False,False,True,False,False,True,False,False,False
3143,9960,True,False,True,False,True,False,False,True,False,False,True,False,False,False


Patients with only a bowel injury

In [11]:
bowel_injury_only  = df[(df['extravasation_healthy'] == True) & (df['bowel_injury'] == True) & 
                                           (df['kidney_healthy'] == True) & (df['liver_healthy'] == True) &
                                            (df['spleen_healthy'] == True)]
print(f'There are {bowel_injury_only.shape[0]} such rows')                                           
bowel_injury_only  

There are 29 such rows


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
5,10065,False,True,True,False,True,False,False,True,False,False,True,False,False,True
50,10929,False,True,True,False,True,False,False,True,False,False,True,False,False,True
67,11335,False,True,True,False,True,False,False,True,False,False,True,False,False,True
119,12332,False,True,True,False,True,False,False,True,False,False,True,False,False,True
152,12951,False,True,True,False,True,False,False,True,False,False,True,False,False,True
176,13403,False,True,True,False,True,False,False,True,False,False,True,False,False,True
546,19763,False,True,True,False,True,False,False,True,False,False,True,False,False,True
558,19914,False,True,True,False,True,False,False,True,False,False,True,False,False,True
936,27196,False,True,True,False,True,False,False,True,False,False,True,False,False,True
1057,29407,False,True,True,False,True,False,False,True,False,False,True,False,False,True


Patients with only an extravasation injury

In [12]:
extravasation_injury_only = df[(df['extravasation_injury'] == True) & (df['bowel_healthy'] == True) & 
                                           (df['kidney_healthy'] == True) & (df['liver_healthy'] == True) &
                                            (df['spleen_healthy'] == True)]
print(f'There are {extravasation_injury_only.shape[0]} such rows')                                             
extravasation_injury_only 

There are 97 such rows


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
22,10292,True,False,False,True,True,False,False,True,False,False,True,False,False,True
28,10494,True,False,False,True,True,False,False,True,False,False,True,False,False,True
57,11044,True,False,False,True,True,False,False,True,False,False,True,False,False,True
70,11379,True,False,False,True,True,False,False,True,False,False,True,False,False,True
76,11474,True,False,False,True,True,False,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999,7026,True,False,False,True,True,False,False,True,False,False,True,False,False,True
3025,7411,True,False,False,True,True,False,False,True,False,False,True,False,False,True
3051,7944,True,False,False,True,True,False,False,True,False,False,True,False,False,True
3122,9528,True,False,False,True,True,False,False,True,False,False,True,False,False,True


Patients with only a low grade kidney injury

In [13]:
kidney_low_only = df[(df['extravasation_healthy'] == True) & (df['bowel_healthy'] == True) & 
                                           (df['kidney_low'] == True) & (df['liver_healthy'] == True) &
                                            (df['spleen_healthy'] == True)]
print(f'There are {kidney_low_only.shape[0]} such rows')                                            
kidney_low_only 

There are 64 such rows


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
63,11222,True,False,True,False,False,True,False,True,False,False,True,False,False,True
69,11378,True,False,True,False,False,True,False,True,False,False,True,False,False,True
201,1381,True,False,True,False,False,True,False,True,False,False,True,False,False,True
218,14281,True,False,True,False,False,True,False,True,False,False,True,False,False,True
267,15057,True,False,True,False,False,True,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2851,63226,True,False,True,False,False,True,False,True,False,False,True,False,False,True
3040,7657,True,False,True,False,False,True,False,True,False,False,True,False,False,True
3045,7754,True,False,True,False,False,True,False,True,False,False,True,False,False,True
3115,9424,True,False,True,False,False,True,False,True,False,False,True,False,False,True


Patients with only a high grade kidney injury

In [14]:
kidney_high_only = df[(df['extravasation_healthy'] == True) & (df['bowel_healthy'] == True) & 
                                           (df['kidney_high'] == True) & (df['liver_healthy'] == True) &
                                            (df['spleen_healthy'] == True)]
print(f'There are {kidney_high_only.shape[0]} such rows')                                            
kidney_high_only 

There are 18 such rows


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
161,13098,True,False,True,False,False,False,True,True,False,False,True,False,False,True
234,1452,True,False,True,False,False,False,True,True,False,False,True,False,False,True
499,19050,True,False,True,False,False,False,True,True,False,False,True,False,False,True
1064,29511,True,False,True,False,False,False,True,True,False,False,True,False,False,True
1127,30613,True,False,True,False,False,False,True,True,False,False,True,False,False,True
1375,35168,True,False,True,False,False,False,True,True,False,False,True,False,False,True
1411,35794,True,False,True,False,False,False,True,True,False,False,True,False,False,True
1413,3580,True,False,True,False,False,False,True,True,False,False,True,False,False,True
1661,40473,True,False,True,False,False,False,True,True,False,False,True,False,False,True
1685,40912,True,False,True,False,False,False,True,True,False,False,True,False,False,True


Patients with only a low grade spleen injury

In [15]:
spleen_low_only = df[(df['extravasation_healthy'] == True) & (df['bowel_healthy'] == True) & 
                                           (df['kidney_healthy'] == True) & (df['liver_healthy'] == True) &
                                            (df['spleen_low'] == True)]
print(f'There are {spleen_low_only.shape[0]} such rows')  
spleen_low_only

There are 146 such rows


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
4,10051,True,False,True,False,True,False,False,True,False,False,False,True,False,True
10,10127,True,False,True,False,True,False,False,True,False,False,False,True,False,True
36,1060,True,False,True,False,True,False,False,True,False,False,False,True,False,True
37,10683,True,False,True,False,True,False,False,True,False,False,False,True,False,True
55,11021,True,False,True,False,True,False,False,True,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3058,8094,True,False,True,False,True,False,False,True,False,False,False,True,False,True
3095,8964,True,False,True,False,True,False,False,True,False,False,False,True,False,True
3096,8978,True,False,True,False,True,False,False,True,False,False,False,True,False,True
3098,902,True,False,True,False,True,False,False,True,False,False,False,True,False,True


Patients with only a high grade spleen injury

In [16]:
spleen_high_only = df[(df['extravasation_healthy'] == True) & (df['bowel_healthy'] == True) & 
                                           (df['kidney_healthy'] == True) & (df['liver_healthy'] == True) &
                                            (df['spleen_high'] == True)]
print(f'There are {spleen_high_only.shape[0]} such rows')                                               
spleen_high_only    

There are 81 such rows


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
11,10132,True,False,True,False,True,False,False,True,False,False,False,False,True,True
23,10295,True,False,True,False,True,False,False,True,False,False,False,False,True,True
29,10503,True,False,True,False,True,False,False,True,False,False,False,False,True,True
49,10925,True,False,True,False,True,False,False,True,False,False,False,False,True,True
249,14784,True,False,True,False,True,False,False,True,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2928,64632,True,False,True,False,True,False,False,True,False,False,False,False,True,True
2995,6990,True,False,True,False,True,False,False,True,False,False,False,False,True,True
3022,7369,True,False,True,False,True,False,False,True,False,False,False,False,True,True
3145,9980,True,False,True,False,True,False,False,True,False,False,False,False,True,True


Patients with only a low grade liver injury

In [17]:
liver_low_only = df[(df['extravasation_healthy'] == True) & (df['bowel_healthy'] == True) & 
                                           (df['kidney_healthy'] == True) & (df['liver_low'] == True) &
                                            (df['spleen_healthy'] == True)]
print(f'There are {liver_low_only.shape[0]} such rows')                                              
liver_low_only 

There are 178 such rows


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
12,10163,True,False,True,False,True,False,False,False,True,False,True,False,False,True
25,10430,True,False,True,False,True,False,False,False,True,False,True,False,False,True
111,12198,True,False,True,False,True,False,False,False,True,False,True,False,False,True
121,12354,True,False,True,False,True,False,False,False,True,False,True,False,False,True
126,12451,True,False,True,False,True,False,False,False,True,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3048,7819,True,False,True,False,True,False,False,False,True,False,True,False,False,True
3056,8054,True,False,True,False,True,False,False,False,True,False,True,False,False,True
3080,8621,True,False,True,False,True,False,False,False,True,False,True,False,False,True
3101,9081,True,False,True,False,True,False,False,False,True,False,True,False,False,True


Patients with only a high grade liver injury

In [18]:
liver_high_only = df[(df['extravasation_healthy'] == True) & (df['bowel_healthy'] == True) & 
                                           (df['kidney_healthy'] == True) & (df['liver_high'] == True) &
                                            (df['spleen_healthy'] == True)]
print(f'There are {liver_high_only.shape[0]} such rows')                                              
liver_high_only 

There are 31 such rows


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
120,12347,True,False,True,False,True,False,False,False,False,True,True,False,False,True
206,13921,True,False,True,False,True,False,False,False,False,True,True,False,False,True
372,16776,True,False,True,False,True,False,False,False,False,True,True,False,False,True
392,17190,True,False,True,False,True,False,False,False,False,True,True,False,False,True
422,17806,True,False,True,False,True,False,False,False,False,True,True,False,False,True
461,18407,True,False,True,False,True,False,False,False,False,True,True,False,False,True
494,1900,True,False,True,False,True,False,False,False,False,True,True,False,False,True
534,1960,True,False,True,False,True,False,False,False,False,True,True,False,False,True
564,20118,True,False,True,False,True,False,False,False,False,True,True,False,False,True
566,20135,True,False,True,False,True,False,False,False,False,True,True,False,False,True


# <div style="color:white;display:inline-block;border-radius:5px;background-color:#FF7F50;font-family:Nexa;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:80%;letter-spacing:0.5px;margin:0"><b> </b> Dataset preparation and preprocessing</p></div>

In [19]:
root_dir =  '/kaggle/input/rsna-2023-abdominal-trauma-detection'
working_dir = '/kaggle/working'
# method to help me calculate the number of dcm images for each scan.
def count_dcm(patient_id):
    nested_folders = os.listdir(os.path.join(root_dir, f"train_images/{patient_id}"))
    
    for nested_folder in nested_folders:
        path_to_nested_folder = os.path.join(root_dir, f"train_images/{patient_id}", nested_folder)
        number_of_dcm_images = int(len(glob(path_to_nested_folder + '/*')))
        print(f'Series number: {nested_folder}, Number of dcm images: {number_of_dcm_images}')
        
# method to help me cleanup a specified directory.
def wipe_directory(directory_path):
    shutil.rmtree(directory_path)

Adding modality attribute to the training images and convert to nifti.

In [20]:
# list_of_training_images = glob(os.path.join(root_dir, "train_images/*"))

bowel_injury_files = ['10065', '10929', '11335', '35022', '45303', '13403', '19763',
                     '32541', '56441', '58465', '53348', '41050', '29407', '27196',
                     '53581', '7482', '56981', '61399', '32011', '43059']

extravasation_injury_files = ['10292', '10494', '11044', '11379', '11474', '7026',
                             '7411', '7944', '9528', '9632', '12210', '12875', '23029',
                             '2602', '21325', '2209', '12192', '15188', '1675', '18779']

kidney_injury_files = ['13098', '1452','19050', '29511', '30613', '35168', '35794', '3580',
                       '40473', '40912', '48540', '49044', '61005', '54190', '60934', '57981',
                        '11222', '11378', '1381', '14281', '15057', '63226', '7657', '7754',
                        '9424', '26239', '2629', '28213', '25231', '16588', '26164',
                        '16963']
liver_injury_files = ['12347', '13921', '16776', '17190', '17806', '1900', '1960', '20118',
                     '20135', '2937', '3568', '34401', '57988', '4813', '55240', '21050', '40633',
                    '10163', '10430', '12198', '1278', '12967', '13563', '13698', '14388',
                    '15141', '15480', '16100', '15385', '23415', '24461', '25306',
                    '25957']
spleen_injury_files = ['10132', '10295', '10503', '10925', '14784', '64632', '6990', '7369',
                       '9980','17178', '18001', '15185', '22595', '27303', '31220', '27575',
                        '10051', '10127', '1060', '10683', '11021', '8094', '8964', '902', 
                        '11664', '11834', '12698', '13517', '11046', '14178', '15089', '16343']
no_injury_files = ['10005', '10007', '10026', '10082', '10104', '9961', '9860', '9951', '9835',
                  '9961', '14251', '14065', '14063', '42924', '46003', '48508', '50805', '50759',
                  '1144', '11030', '10565', '11452', '11770', '11793', '27910', '30396', '32565',
                  '15398']

# Each file corresponds to a unique patient in the "test_images" directory.
test_files_used = ['48843', '50046', '63706']

def create_nifti_files(files_used, folder_to_check):
    for patient_file in files_used:
        nested_folders = os.listdir(os.path.join(root_dir, f"{folder_to_check}/{patient_file}"))
        
        for nested_folder in nested_folders:
            path_to_nested_folder = os.path.join(root_dir, f"{folder_to_check}/{patient_file}", nested_folder)
            files = glob(path_to_nested_folder + '/*')
            number_of_files = int(len(files))
            
            if number_of_files < 3:
                settings.disable_validate_slicecount()
                
            # Adding modularity attribute
            for file in sorted(files):
                series_number = os.path.splitext(os.path.basename(os.path.normpath(file)))[0]
                ct = pydicom.dcmread(file)
                ct.Modality = 'CT'
                # make directory in working dir to store new files
                output_dir = os.path.join(working_dir, 'dicom', nested_folder)
                os.makedirs(output_dir, exist_ok=True)
                # save files 
                ct.save_as(os.path.join(output_dir, f'{series_number}.dcm'))
            # make directory to store nifti files
            nifti_dir = os.path.join(working_dir, 'nifti')
            os.makedirs(nifti_dir, exist_ok=True)
            # convert the files in the dicom folder to nifti file format.
            containing_dcm = os.path.join(working_dir, 'dicom', nested_folder)
            dicom2nifti.dicom_series_to_nifti(containing_dcm, os.path.join(nifti_dir, nested_folder + '.nii.gz'))
            # delete dicom files after generating nifti files.
            shutil.rmtree(os.path.join(working_dir, 'dicom', nested_folder))
            



train, test and evaluation files

In [21]:

data_path = os.sep.join(['/kaggle', 'working', 'nifti'])

# injuries
injury_names =['bowel', 'extravasation', 'kidney', 'liver', 'spleen']

bowel_train_images = [
    # no bowel injury, class 0
    '27573.nii.gz',
    '53234.nii.gz',
    '64202.nii.gz',
    '1119.nii.gz',
    '55528.nii.gz',
    '44849.nii.gz',
    '29995.nii.gz',
    '29700.nii.gz',
    '2731.nii.gz',
    '33303.nii.gz',
    '64587.nii.gz',
    '18685.nii.gz',
    '40374.nii.gz',
    '47578.nii.gz',
    '11416.nii.gz',
    '60509.nii.gz',
    '5576.nii.gz',
    '2003.nii.gz',
    '42932.nii.gz',
    '18667.nii.gz',
    # bowel injury, class 1
    '16882.nii.gz',
    '44685.nii.gz',
    '65185.nii.gz',
    '46118.nii.gz',
    '21078.nii.gz',
    '39434.nii.gz',
    '64814.nii.gz',
    '60138.nii.gz',
    '13768.nii.gz',
    '17182.nii.gz',
    '17217.nii.gz',
    '37324.nii.gz',
    '38658.nii.gz',
    '62600.nii.gz',
    '12288.nii.gz',
    '35173.nii.gz',
    '29294.nii.gz',
    '6439.nii.gz',
    '34169.nii.gz',
    '44615.nii.gz'
]
bowel_train_images = [os.sep.join([data_path, f]) for f in bowel_train_images]  
bowel_train_labels = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                               1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=np.int64)
bowel_train_labels = torch.nn.functional.one_hot(torch.as_tensor(bowel_train_labels), 2).float()

bowel_test_images = [
    # no bowel injury, class 0
    '29591.nii.gz',
    '47210.nii.gz',
    '13560.nii.gz',
    '60588.nii.gz',
    # bowel injury, class 1
    '63574.nii.gz',
    '21228.nii.gz',
    '24276.nii.gz',
    '4486.nii.gz'
]

bowel_test_images = [os.sep.join([data_path, f]) for f in bowel_test_images]
bowel_test_labels = np.array([0, 0, 0, 0, 1, 1, 1, 1], dtype=np.int64)
bowel_test_labels = torch.nn.functional.one_hot(torch.as_tensor(bowel_test_labels), 2).float()

bowel_evaluation_images = [
    # no bowel injury, class 0
    '54756.nii.gz',
    '999.nii.gz',
    '26866.nii.gz',
    '37533.nii.gz',
    # bowel injury, class 1
    '15586.nii.gz',
    '49474.nii.gz',
    '6027.nii.gz',
    '55120.nii.gz'
]
bowel_evaluation_images = [os.sep.join([data_path, f]) for f in bowel_evaluation_images]  
bowel_evaluation_labels = np.array([0, 0, 0, 0, 1, 1, 1, 1], dtype=np.int64)
bowel_evaluation_labels = torch.nn.functional.one_hot(torch.as_tensor(bowel_evaluation_labels), 2).float()


extravasation_train_images = [
    # no extravasation injury, class 0
    '27573.nii.gz',
    '53234.nii.gz',
    '64202.nii.gz',
    '1119.nii.gz',
    '55528.nii.gz',
    '44849.nii.gz',
    '29995.nii.gz',
    '29700.nii.gz',
    '2731.nii.gz',
    '33303.nii.gz',
    '64587.nii.gz',
    '18685.nii.gz',
    '40374.nii.gz',
    '47578.nii.gz',
    '11416.nii.gz',
    '60509.nii.gz',
    '5576.nii.gz',
    '2003.nii.gz',
    '42932.nii.gz',
    '18667.nii.gz',
    # extravasation injury, class 1
    '14945.nii.gz',
    '29647.nii.gz',
    '65369.nii.gz',
    '56431.nii.gz',
    '15870.nii.gz',
    '28676.nii.gz',
    '1126.nii.gz',
    '38305.nii.gz',
    '23209.nii.gz',
    '64737.nii.gz',
    '12196.nii.gz',
    '21576.nii.gz',
    '29304.nii.gz',
    '35443.nii.gz',
    '42688.nii.gz',
    '1989.nii.gz',
    '29661.nii.gz',
    '4654.nii.gz',
    '25323.nii.gz',
    '54527.nii.gz'
]
extravasation_train_images = [os.sep.join([data_path, f]) for f in extravasation_train_images]  
extravasation_train_labels = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 
                                       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=np.int64)
extravasation_train_labels = torch.nn.functional.one_hot(torch.as_tensor(extravasation_train_labels), 2).float()
extravasation_test_images = [
    # no extravasation injury, class 0
    '29591.nii.gz',
    '47210.nii.gz',
    '13560.nii.gz',
    '60588.nii.gz',
    # extravasation injury, class 1
    '37630.nii.gz',
    '53257.nii.gz',
    '3750.nii.gz',
    '47364.nii.gz'
]

extravasation_test_images = [os.sep.join([data_path, f]) for f in extravasation_test_images]
extravasation_test_labels = np.array([0, 0, 0, 0, 1, 1, 1, 1], dtype=np.int64)
extravasation_test_labels = torch.nn.functional.one_hot(torch.as_tensor(extravasation_test_labels), 2).float()

extravasation_evaluation_images = [
    # no extravasation injury, class 0
    '54756.nii.gz',
    '999.nii.gz',
    '26866.nii.gz',
    '37533.nii.gz',
    # extravasation injury, class 1
    '21338.nii.gz',
    '45638.nii.gz',
    '60750.nii.gz',
    '62696.nii.gz'
]
extravasation_evaluation_images = [os.sep.join([data_path, f]) for f in extravasation_evaluation_images]  
extravasation_evaluation_labels = np.array([0, 0, 0, 0, 1, 1, 1, 1], dtype=np.int64)
extravasation_evaluation_labels = torch.nn.functional.one_hot(torch.as_tensor(extravasation_evaluation_labels), 2).float()


kidney_train_images = [
    # no kidney injury, class 0
    '27573.nii.gz',
    '53234.nii.gz',
    '64202.nii.gz',
    '1119.nii.gz',
    '55528.nii.gz',
    '44849.nii.gz',
    '29995.nii.gz',
    '29700.nii.gz',
    '2731.nii.gz',
    '33303.nii.gz',
    '64587.nii.gz',
    '18685.nii.gz',
    '40374.nii.gz',
    # Low grade kidney injury, class 1
    '39373.nii.gz',
    '53429.nii.gz',
    '10523.nii.gz',
    '6366.nii.gz',
    '43200.nii.gz',
    '28997.nii.gz',
    '13131.nii.gz',
    '49511.nii.gz',
    '6910.nii.gz', 
    '28566.nii.gz',
    '42511.nii.gz',
    '59048.nii.gz',
    '44803.nii.gz',
    # High grade kidney injury, class 2
    '49881.nii.gz',
    '8188.nii.gz',
    '17227.nii.gz',
    '57250.nii.gz',
    '21235.nii.gz',
    '43565.nii.gz',
    '16724.nii.gz',
    '22173.nii.gz',
    '36927.nii.gz',
    '21227.nii.gz',
    '3244.nii.gz',
    '23993.nii.gz',
    '25886.nii.gz'
]
kidney_train_images = [os.sep.join([data_path, f]) for f in kidney_train_images]  
kidney_train_labels = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
                                2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=np.int64)
kidney_train_labels = torch.nn.functional.one_hot(torch.as_tensor(kidney_train_labels), 3).float()

kidney_test_images = [
    # no kidney injury, class 0
    '29591.nii.gz',
    '47210.nii.gz',
    '13560.nii.gz',
    '60588.nii.gz',
    # low grade kidney injury, class 1
    '44771.nii.gz',
    '50158.nii.gz',
    '38081.nii.gz',
    '45575.nii.gz',
    # high grade kidney injury, class 2
    '12274.nii.gz',
    '41008.nii.gz',
    '36139.nii.gz',
    '77.nii.gz'
]

kidney_test_images = [os.sep.join([data_path, f]) for f in kidney_test_images]
kidney_test_labels = np.array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2], dtype=np.int64)
kidney_test_labels = torch.nn.functional.one_hot(torch.as_tensor(kidney_test_labels), 3).float()

kidney_evaluation_images = [
    # no kidney injury, class 0
    '54756.nii.gz',
    '999.nii.gz',
    '26866.nii.gz',
    '37533.nii.gz',
    # low grade kidney injury, class 1
    '31273.nii.gz',
    '63028.nii.gz',
    '56068.nii.gz',
    '56629.nii.gz',
    # high grade kidney injury, class 2
    '32893.nii.gz',
    '17269.nii.gz',
    '6223.nii.gz',
    '30805.nii.gz',
]
kidney_evaluation_images = [os.sep.join([data_path, f]) for f in kidney_evaluation_images]  
kidney_evaluation_labels = np.array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2], dtype=np.int64)
kidney_evaluation_labels = torch.nn.functional.one_hot(torch.as_tensor(kidney_evaluation_labels), 3).float()

liver_train_images = [
    # no liver injury, class 0
    '27573.nii.gz',
    '53234.nii.gz',
    '64202.nii.gz',
    '1119.nii.gz',
    '55528.nii.gz',
    '44849.nii.gz',
    '29995.nii.gz',
    '29700.nii.gz',
    '2731.nii.gz',
    '33303.nii.gz',
    '64587.nii.gz',
    '18685.nii.gz',
    '40374.nii.gz',
    # low grade liver injury, class 1
    '6738.nii.gz',
    '6417.nii.gz',
    '47009.nii.gz',
    '27583.nii.gz',
    '781.nii.gz',
    '34096.nii.gz',
    '17137.nii.gz',
    '12070.nii.gz',
    '11550.nii.gz'
    '34040.nii.gz',
    '12312.nii.gz',
    '39579.nii.gz',
    '47601.nii.gz',
    # High grade liver injury, class 2
    '14440.nii.gz',
    '54841.nii.gz',
    '13285.nii.gz',
    '11403.nii.gz',
    '1952.nii.gz',
    '1796.nii.gz',
    '23671.nii.gz',
    '48371.nii.gz',
    '41002.nii.gz',
    '13901.nii.gz',
    '16384.nii.gz',
    '17196.nii.gz',
    '63081.nii.gz'
]
liver_train_images = [os.sep.join([data_path, f]) for f in liver_train_images]  
liver_train_labels = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                               2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=np.int64)
liver_train_labels = torch.nn.functional.one_hot(torch.as_tensor(liver_train_labels), 3).float()

liver_test_images = [
    # no liver injury, class 0
    '29591.nii.gz',
    '47210.nii.gz',
    '13560.nii.gz',
    '60588.nii.gz',
    # low grade liver injury, class 1
    '17557.nii.gz',
    '7650.nii.gz',
    '53750.nii.gz',
    '40492.nii.gz',
    # high grade liver injury, class 2
    '8242.nii.gz',
    '6536.nii.gz',
    '2473.nii.gz',
    '6871.nii.gz'
    
]

liver_test_images = [os.sep.join([data_path, f]) for f in liver_test_images]
liver_test_labels = np.array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2], dtype=np.int64)
liver_test_labels = torch.nn.functional.one_hot(torch.as_tensor(liver_test_labels), 3).float()

liver_evaluation_images = [
    # no liver injury, class 0
    '54756.nii.gz',
    '999.nii.gz',
    '26866.nii.gz',
    '37533.nii.gz',
    # low grade liver injury, class 1
    '55621.nii.gz',
    '52122.nii.gz',
    '52259.nii.gz',
    '56685.nii.gz',
    # high grade liver injury, class 1
    '45401.nii.gz',
    '12584.nii.gz',
    '36073.nii.gz',
    '23522.nii.gz'
]
liver_evaluation_images = [os.sep.join([data_path, f]) for f in liver_evaluation_images]  
liver_evaluation_labels = np.array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2], dtype=np.int64)
liver_evaluation_labels = torch.nn.functional.one_hot(torch.as_tensor(liver_evaluation_labels), 3).float()


spleen_train_images = [
    # no spleen injury, class 0
    '27573.nii.gz',
    '53234.nii.gz',
    '64202.nii.gz',
    '1119.nii.gz',
    '55528.nii.gz',
    '44849.nii.gz',
    '29995.nii.gz',
    '29700.nii.gz',
    '2731.nii.gz',
    '33303.nii.gz',
    '64587.nii.gz',
    '18685.nii.gz',
    '40374.nii.gz',
    # low grade spleen injury, class 1
    '64776.nii.gz',
    '60031.nii.gz',
    '37456.nii.gz',
    '15777.nii.gz',
    '50172.nii.gz',
    '12625.nii.gz',
    '58443.nii.gz',
    '59240.nii.gz',
    '52163.nii.gz',
    '19484.nii.gz',
    '29137.nii.gz',
    '57414.nii.gz',
    '14324.nii.gz',
    # high grade spleen injury, class 2
    '1157.nii.gz',
    '40214.nii.gz',
    '10273.nii.gz',
    '52057.nii.gz',
    '10518.nii.gz',
    '43936.nii.gz',
    '60534.nii.gz',
    '40466.nii.gz',
    '22256.nii.gz',
    '4930.nii.gz',
    '24164.nii.gz',
    '52455.nii.gz',
    '54600.nii.gz'

]
spleen_train_images = [os.sep.join([data_path, f]) for f in spleen_train_images]  
spleen_train_labels = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
                                2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=np.int64)
spleen_train_labels = torch.nn.functional.one_hot(torch.as_tensor(spleen_train_labels), 3).float()

spleen_test_images = [
    # no spleen injury, class 0
    '29591.nii.gz',
    '47210.nii.gz',
    '13560.nii.gz',
    '60588.nii.gz',
    # low grade spleen injury, class 1
    '2180.nii.gz',
    '29529.nii.gz',
    '17000.nii.gz',
    '53380.nii.gz',
    # high grade spleen injury, class 2
    '4816.nii.gz',
    '49250.nii.gz',
    '54651.nii.gz',
    '32849.nii.gz'
]

spleen_test_images = [os.sep.join([data_path, f]) for f in spleen_test_images]
spleen_test_labels = np.array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2], dtype=np.int64)
spleen_test_labels = torch.nn.functional.one_hot(torch.as_tensor(spleen_test_labels), 3).float()

spleen_evaluation_images = [
    # no spleen injury, class 0
    '54756.nii.gz',
    '999.nii.gz',
    '26866.nii.gz',
    '37533.nii.gz',
    # low grade spleen injury, class 1
    '3716.nii.gz',
    '26599.nii.gz',
    '17486.nii.gz',
    '5060.nii.gz',
    # high grade spleen injury, class 2
    '10410.nii.gz',
    '19713.nii.gz',
    '12055.nii.gz',
    '22975.nii.gz'
]
spleen_evaluation_images = [os.sep.join([data_path, f]) for f in spleen_evaluation_images]  
spleen_evaluation_labels = np.array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2], dtype=np.int64)
spleen_evaluation_labels = torch.nn.functional.one_hot(torch.as_tensor(spleen_evaluation_labels), 3).float()

# image transformations
transforms = Compose([
    ScaleIntensityRange(a_min=-200, a_max=200, b_min=0.0, b_max=1.0, clip=True),
    EnsureChannelFirst(),
    Orientation(axcodes='RAS'),
    Resize((128,128,64))
])

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#FF7F50;font-family:Nexa;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:80%;letter-spacing:0.5px;margin:0"><b> </b> Training DenseNet264 model for 3D classification</p></div>

In [22]:
# train the model.
def train(injury_name):

    set_determinism(seed = 42)
    
    create_nifti_files(no_injury_files, 'train_images')
    
    if injury_name == 'bowel':
        create_nifti_files(bowel_injury_files, 'train_images')
        # create a training data loader
        train_ds = ImageDataset(image_files=bowel_train_images, labels=bowel_train_labels, transform=transforms)
        train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2, pin_memory=pin_memory)
        # create a testing data loader
        test_ds = ImageDataset(image_files=bowel_test_images, labels=bowel_test_labels, transform=transforms)
        test_loader = DataLoader(test_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)

    elif injury_name == 'extravasation':
        create_nifti_files(extravasation_injury_files, 'train_images')
        # create a training data loader
        train_ds = ImageDataset(image_files=extravasation_train_images, labels=extravasation_train_labels, transform=transforms)
        train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2, pin_memory=pin_memory)
        # create a testing data loader
        test_ds = ImageDataset(image_files=extravasation_test_images, labels=extravasation_test_labels, transform=transforms)
        test_loader = DataLoader(test_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)

    elif injury_name == 'kidney':
        create_nifti_files(kidney_injury_files, 'train_images')
        # create a training data loader
        train_ds = ImageDataset(image_files=kidney_train_images, labels=kidney_train_labels, transform=transforms)
        train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2, pin_memory=pin_memory)
        # create a testing data loader
        test_ds = ImageDataset(image_files=kidney_test_images, labels=kidney_test_labels, transform=transforms)
        test_loader = DataLoader(test_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)

    elif injury_name == 'liver':
        create_nifti_files(liver_injury_files, 'train_images')
        # create a training data loader
        train_ds = ImageDataset(image_files=liver_train_images, labels=liver_train_labels, transform=transforms)
        train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2, pin_memory=pin_memory)
        # create a testing data loader
        test_ds = ImageDataset(image_files=liver_test_images, labels=liver_test_labels, transform=transforms)
        test_loader = DataLoader(test_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)

    else:
        create_nifti_files(spleen_injury_files, 'train_images')
        # create a training data loader
        train_ds = ImageDataset(image_files=spleen_train_images, labels=spleen_train_labels, transform=transforms)
        train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2, pin_memory=pin_memory)
        # create a testing data loader
        test_ds = ImageDataset(image_files=spleen_test_images, labels=spleen_test_labels, transform=transforms)
        test_loader = DataLoader(test_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)
      
    # Create DenseNet121, CrossEntropyLoss and Adam optimizer
    if injury_name == 'bowel' or injury_name == 'extravasation':
        model = monai.networks.nets.DenseNet264(spatial_dims=3, in_channels=1, out_channels=2).to(device)
    else:
        model = monai.networks.nets.DenseNet264(spatial_dims=3, in_channels=1, out_channels=3).to(device)
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), 1e-5)
    
    # initial metrics
    test_interval = 2
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = []
    metric_values = []
    writer = SummaryWriter()
    max_epochs = 0

    if injury_name == 'bowel' or injury_name == 'extravasation':
        max_epochs = 20
    else:
        max_epochs = 40
    
    for epoch in range(max_epochs):
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")
        model.train()
        epoch_loss = 0
        step = 0
        
        for batch_data in train_loader:
            step += 1
            inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_len = len(train_ds) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")


            
        if (epoch + 1) % test_interval == 0:
            model.eval()

            num_correct = 0.0
            metric_count = 0
            for test_data in test_loader:
                test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
                with torch.no_grad():
                    test_outputs = model(test_images)
                    value = torch.eq(test_outputs.argmax(dim=1), test_labels.argmax(dim=1))
                    metric_count += len(value)
                    num_correct += value.sum().item()
                    
            metric = num_correct / metric_count
            metric_values.append(metric)

            # saving the model with the best metrics.
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                best_metric_model_dir = os.path.join(working_dir, 'best_metric')
                os.makedirs(best_metric_model_dir, exist_ok=True)
                save_path = f'/kaggle/working/best_metric/best_metric_model_for_{injury_name}.pth'
                torch.save(model.state_dict(), save_path)
                print(f"Saved new best metric model for {injury_name} injuries")

                print(f"Current epoch: {epoch+1} current accuracy: {metric:.4f} ")
                print(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}")
                writer.add_scalar("test_accuracy", metric, epoch + 1)

    print(f'Training completed for model concerning {injury_name} injuries.')
    print(f"Best metric is: {best_metric:.4f} at epoch: {best_metric_epoch}")
    writer.close()
    # Delete nifti files after training
    wipe_directory('/kaggle/working/nifti')

#for injury_name in injury_names:
    #train(injury_name)

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#FF7F50;font-family:Nexa;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:80%;letter-spacing:0.5px;margin:0"><b> </b> Model evaluation/ Validation</p></div>

In [23]:
# evaluate the models
def evaluate(injury_name):
    create_nifti_files(no_injury_files, 'train_images')
    # create a evaluation data loader
    if injury_name == 'bowel':
        create_nifti_files(bowel_injury_files, 'train_images')
        val_ds = ImageDataset(image_files=bowel_evaluation_images, labels=bowel_evaluation_labels, transform=transforms)
        val_loader = DataLoader(val_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)
    elif injury_name == 'extravasation':
        create_nifti_files(extravasation_injury_files, 'train_images')
        val_ds = ImageDataset(image_files=extravasation_evaluation_images, labels=extravasation_evaluation_labels, transform=transforms)
        val_loader = DataLoader(val_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)
    elif injury_name == 'kidney':
        create_nifti_files(kidney_injury_files, 'train_images')
        val_ds = ImageDataset(image_files=kidney_evaluation_images, labels=kidney_evaluation_labels, transform=transforms)
        val_loader = DataLoader(val_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)
    elif injury_name == 'liver':
        create_nifti_files(liver_injury_files, 'train_images')
        val_ds = ImageDataset(image_files=liver_evaluation_images, labels=liver_evaluation_labels, transform=transforms)
        val_loader = DataLoader(val_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)
    else:
        create_nifti_files(spleen_injury_files, 'train_images')
        val_ds = ImageDataset(image_files=spleen_evaluation_images, labels=spleen_evaluation_labels, transform=transforms)
        val_loader = DataLoader(val_ds, batch_size=1, num_workers=2, pin_memory=pin_memory)
        
    # Create DenseNet121
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if injury_name == 'bowel' or injury_name == 'extravasation':
        model = monai.networks.nets.DenseNet264(spatial_dims=3, in_channels=1, out_channels=2).to(device)
    else:
        model = monai.networks.nets.DenseNet264(spatial_dims=3, in_channels=1, out_channels=3).to(device)

    save_path = f'/kaggle/working/best_metric/best_metric_model_for_{injury_name}.pth'
    model.load_state_dict(torch.load(save_path))
    model.eval()
    with torch.no_grad():
        num_correct = 0.0
        metric_count = 0
        evaluation_dir = os.path.join(working_dir, f'{injury_name}_evaluation')
        os.makedirs(evaluation_dir, exist_ok=True)
        saver = CSVSaver(output_dir=evaluation_dir)
        for val_data in val_loader:
            val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
            val_outputs = model(val_images).argmax(dim=1)
            value = torch.eq(val_outputs, val_labels)
            metric_count += len(value)
            num_correct += value.sum().item()
            saver.save_batch(val_outputs, val_images.meta)
        metric = num_correct / metric_count
        print(f"Evaluation metric for the {injury_name} model is:", metric)
        saver.finalize()
    
    wipe_directory('/kaggle/working/nifti')
    
#for injury_name in injury_names:
    #evaluate(injury_name)

# <div style="color:white;display:inline-block;border-radius:5px;background-color:#FF7F50;font-family:Nexa;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:80%;letter-spacing:0.5px;margin:0"><b> </b> Classifying test images</p></div>

In [24]:
def get_predictions():
    create_nifti_files(test_files_used, 'test_images')
    test_images = [
        '62825.nii.gz',
        '24574.nii.gz',
        '39279.nii.gz'
    ]
    test_images = [os.sep.join([data_path, f]) for f in test_images]
    
    for injury_name in injury_names:
        # Create DenseNet264 model
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if injury_name == 'bowel' or injury_name == 'extravasation':
            model = monai.networks.nets.DenseNet264(spatial_dims=3, in_channels=1, out_channels=2).to(device)
        else:
            model = monai.networks.nets.DenseNet264(spatial_dims=3, in_channels=1, out_channels=3).to(device)

        save_path = f'/kaggle/working/best_metric/best_metric_model_for_{injury_name}.pth'
        model.load_state_dict(torch.load(save_path))
        model.eval()
        
        for test_image in test_images:
            print(test_image)



# <div style="color:white;display:inline-block;border-radius:5px;background-color:#FF7F50;font-family:Nexa;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;font-size:80%;letter-spacing:0.5px;margin:0"><b> </b> Generating submission file</p></div>